In [1]:
# !python -m spacy download fr_core_news_md

In [2]:
file = 'lepen_speech_YT.csv'

In [3]:
import pandas as pd
from keybert import KeyBERT
import re
from bertopic import BERTopic
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
nlp = spacy.load('fr_core_news_md')
# nltk.download('wordnet')
# nltk.download('punkt')
stopwords = list(fr_stop)
kw_model = KeyBERT()

In [4]:
df = pd.read_csv(file, encoding = 'UTF-8')

In [5]:
punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@' 

stopwords.extend(['avoir', 'pouvoir', 'devoir'])

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f" 
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', text)

def preprocessing_final(tweet):
    # nlp = spacy.load('fr_core_news_md')
    # To lowercase
    punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@' 
    """function that also performs tokenization, lemmatization and removal of stopwords"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet=deEmojify(tweet)
# Creating a doc with spaCy
    doc = nlp(tweet)
    lemmas = []
    for token in doc:
        lemmas.append(token.lemma_)
    return " ".join([str(x) for x in lemmas if x.isalpha() and x not in stopwords])

In [6]:
df['preprocessed']= df['transcript'].apply(preprocessing_final)

In [7]:
def keyword_extraction(text):
    keywords=[]
    kw = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=stopwords)
    kw.extend(kw_model.extract_keywords(text, keyphrase_ngram_range=(3, 3), stop_words=stopwords, 
                              use_mmr=True, diversity=0.7))
    for (k,v) in kw:
        keywords.append(k)
    
    return ' '.join(list(set(' '.join(keywords).split())))

In [8]:
df['keywords']= df['transcript'].apply(keyword_extraction)

In [9]:
texts = df.keywords.to_list()

In [13]:
topic_model = BERTopic(language="French")
topics, probs = topic_model.fit_transform(texts)

In [14]:
Topic_df = pd.DataFrame.from_dict(topic_model.get_topics())

In [16]:
Topic_df.to_csv(file[:-4]+'_topics.csv', encoding='utf-8')